# 자치구 단위 택배 물동량 데이터 -> 행정구 단위 택배 물동량 데이터 변환
행정동인구/자치구인구 비율을 물동량에 곱함으로써 행정동 단위의 택배 물동량 추정 데이터 만드는 것이 목적.

In [92]:
import pandas as pd

In [96]:
# 1. 데이터 로드
# 택배 물동량 데이터 로드 
# 컬럼 예시: '배송년월일', '수하인_구명', '수하인_구코드', '대분류_착지물동량 가구/인테리어', ...
parcel_df = pd.read_csv("logistics_18_23.csv", encoding='euc-kr', dtype={
    '배송년월일': str,
    '수하인_구명': str,
    '수하인_구코드': int,
    '대분류_착지물동량 가구/인테리어': int,
    '대분류_착지물동량 기타': int,
    '대분류_착지물동량 도서/음반': int,
    '대분류_착지물동량 디지털/가전': int,
    '대분류_착지물동량 생활/건강': int,
    '대분류_착지물동량 스포츠/레저': int,
    '대분류_착지물동량 식품': int,
    '대분류_착지물동량 출산/육아': int,
    '대분류_착지물동량 패션의류': int,
    '대분류_착지물동량 패션잡화': int,
    '대분류_착지물동량 화장품/미용': int
})

# 행정동 비율 데이터 로드 
# 컬럼 예시: '시군구명', '행정동', '행정동인구', '자치구인구', '비율'
ratio_df = pd.read_csv("people.csv", encoding='euc-kr', dtype={
    '시군구명': str,
    '행정동': str,
    '행정동인구': int,
    '자치구인구': int,
    '비율': float
})

In [100]:
# 2. 데이터 병합
# '수하인_구명' (parcel_df)과 '시군구명' (ratio_df)을 기준으로 병합
merged_df = pd.merge(parcel_df, ratio_df, left_on='수하인_구명', right_on='시군구명', how='left')

# 병합 후 일부 자치구명이 매칭되지 않은 경우 확인
if merged_df['행정동'].isnull().any():
    unmatched = merged_df[merged_df['행정동'].isnull()]['수하인_구명'].unique()
    print(f"경고: 다음 자치구명이 행정동 비율 데이터와 매칭되지 않았습니다: {unmatched}")

# 매칭된 행만 필터링 (선택 사항)
merged_df = merged_df[~merged_df['행정동'].isnull()]

# 3. 택배 카테고리 리스트 정의
parcel_categories = [
    '대분류_착지물동량 가구/인테리어',
    '대분류_착지물동량 기타',
    '대분류_착지물동량 도서/음반',
    '대분류_착지물동량 디지털/가전',
    '대분류_착지물동량 생활/건강',
    '대분류_착지물동량 스포츠/레저',
    '대분류_착지물동량 식품',
    '대분류_착지물동량 출산/육아',
    '대분류_착지물동량 패션의류',
    '대분류_착지물동량 패션잡화',
    '대분류_착지물동량 화장품/미용'
]

# 4. 행정동 비율을 적용하여 택배 물동량 분배
for category in parcel_categories:
    merged_df[category + '_행정동'] = merged_df[category] * merged_df['비율']

# 5. 필요한 컬럼만 선택하여 새로운 데이터프레임 생성
# 예시로 '배송년월일', '자치구명', '행정동명', '수하인_구코드'와 분배된 택배 카테고리
neighborhood_df = merged_df.rename(columns={
    '시군구명': '자치구명',
    '행정동': '행정동명'
})[[
    '배송년월일',
    '자치구명',
    '행정동명',
    '수하인_구코드'
] + [category + '_행정동' for category in parcel_categories]]

# 6. 택배 물동량을 정수형으로 변환 (소수점 제거)
neighborhood_df[parcel_categories] = neighborhood_df[[category + '_행정동' for category in parcel_categories]].round().astype(int)

# 필요 없는 '_행정동' 접미사 제거
neighborhood_df = neighborhood_df.rename(columns=lambda x: x.replace('_행정동', '') if '_행정동' in x else x)

# 7. 최종 데이터프레임 확인
print(neighborhood_df.head())

# 8. 새로운 데이터프레임 저장 (필요시)
# neighborhood_df.to_csv('neighborhood_parcel_data.csv', index=False)

      배송년월일 자치구명  행정동명  수하인_구코드  대분류_착지물동량 가구/인테리어  대분류_착지물동량 기타  \
0  20180101  강남구   신사동    11680           5.005868     38.435859   
1  20180101  강남구  논현1동    11680           6.842504     52.537849   
2  20180101  강남구  논현2동    11680           6.663297     51.161864   
3  20180101  강남구  압구정동    11680           8.263896     63.451521   
4  20180101  강남구   청담동    11680           8.061762     61.899509   

   대분류_착지물동량 도서/음반  대분류_착지물동량 디지털/가전  대분류_착지물동량 생활/건강  대분류_착지물동량 스포츠/레저  ...  \
0        24.655337          9.234963        28.481662          2.042624  ...   
1        33.701300         12.623241        38.931490          2.792056  ...   
2        32.818651         12.292634        37.911860          2.718931  ...   
3        40.702061         15.245463        47.018717          3.372049  ...   
4        39.706496         14.872562        45.868648          3.289570  ...   

   대분류_착지물동량 기타  대분류_착지물동량 도서/음반  대분류_착지물동량 디지털/가전  대분류_착지물동량 생활/건강  \
0            38               25       

In [104]:
neighborhood_df.columns

Index(['배송년월일', '자치구명', '행정동명', '수하인_구코드', '대분류_착지물동량 가구/인테리어', '대분류_착지물동량 기타',
       '대분류_착지물동량 도서/음반', '대분류_착지물동량 디지털/가전', '대분류_착지물동량 생활/건강',
       '대분류_착지물동량 스포츠/레저', '대분류_착지물동량 식품', '대분류_착지물동량 출산/육아', '대분류_착지물동량 패션의류',
       '대분류_착지물동량 패션잡화', '대분류_착지물동량 화장품/미용', '대분류_착지물동량 가구/인테리어',
       '대분류_착지물동량 기타', '대분류_착지물동량 도서/음반', '대분류_착지물동량 디지털/가전',
       '대분류_착지물동량 생활/건강', '대분류_착지물동량 스포츠/레저', '대분류_착지물동량 식품',
       '대분류_착지물동량 출산/육아', '대분류_착지물동량 패션의류', '대분류_착지물동량 패션잡화',
       '대분류_착지물동량 화장품/미용'],
      dtype='object')

In [108]:
neighborhood_df.iloc[0:23]

,배송년월일,자치구명,행정동명,수하인_구코드,대분류_착지물동량 가구/인테리어,대분류_착지물동량 기타,대분류_착지물동량 도서/음반,대분류_착지물동량 디지털/가전,대분류_착지물동량 생활/건강,대분류_착지물동량 스포츠/레저,...,대분류_착지물동량 기타,대분류_착지물동량 도서/음반,대분류_착지물동량 디지털/가전,대분류_착지물동량 생활/건강,대분류_착지물동량 스포츠/레저,대분류_착지물동량 식품,대분류_착지물동량 출산/육아,대분류_착지물동량 패션의류,대분류_착지물동량 패션잡화,대분류_착지물동량 화장품/미용
0,20180101,강남구,신사동,11680,5.005868,38.435859,24.655337,9.234963,28.481662,2.042624,...,38,25,9,28,2,39,2,24,14,31
1,20180101,강남구,논현1동,11680,6.842504,52.537849,33.701300,12.623241,38.931490,2.792056,...,53,34,13,39,3,53,2,33,19,42
2,20180101,강남구,논현2동,11680,6.663297,51.161864,32.818651,12.292634,37.911860,2.718931,...,51,33,12,38,3,52,2,32,18,41
3,20180101,강남구,압구정동,11680,8.263896,63.451521,40.702061,15.245463,47.018717,3.372049,...,63,41,15,47,3,64,3,40,23,51
4,20180101,강남구,청담동,11680,8.061762,61.899509,39.706496,14.872562,45.868648,3.289570,...,62,40,15,46,3,63,3,39,22,50
5,20180101,강남구,삼성1동,11680,3.985837,30.603899,19.631393,7.353182,22.678039,1.626405,...,31,20,7,23,2,31,1,19,11,25
6,20180101,강남구,삼성2동,11680,9.825101,75.438709,48.391447,18.125618,55.901439,4.009093,...,75,48,18,56,4,77,3,47,27,61
7,20180101,강남구,대치1동,11680,7.709482,59.194644,37.971414,14.222665,43.864294,3.145823,...,59,38,14,44,3,60,3,37,21,48
8,20180101,강남구,대치2동,11680,12.099263,92.900088,59.592347,22.321054,68.840634,4.937056,...,93,60,22,69,5,94,4,58,33,75
9,20180101,강남구,대치4동,11680,6.090155,46.761190,29.995763,11.235286,34.650882,2.485063,...,47,30,11,35,2,47,2,29,17,38


In [112]:
# 자치구 단위의 원래 물동량 합계
original_sum = parcel_df.groupby('수하인_구명')[parcel_categories].sum().reset_index()

# 행정동 단위로 분배된 물동량 합계
distributed_sum = neighborhood_df.groupby('자치구명')[parcel_categories].sum().reset_index()

# 두 데이터프레임을 비교
comparison_df = pd.merge(original_sum, distributed_sum, left_on='수하인_구명', right_on='자치구명', suffixes=('_original', '_distributed'))

# 차이 계산
for category in parcel_categories:
    comparison_df[f'{category}_difference'] = comparison_df[f'{category}_original'] - comparison_df[f'{category}_distributed']

print(comparison_df.head())

TypeError: '<' not supported between instances of 'str' and 'int'

In [114]:
neighborhood_df.iloc[0]

배송년월일                 20180101
자치구명                       강남구
행정동명                       신사동
수하인_구코드                  11680
대분류_착지물동량 가구/인테리어     5.005868
대분류_착지물동량 기타         38.435859
대분류_착지물동량 도서/음반      24.655337
대분류_착지물동량 디지털/가전      9.234963
대분류_착지물동량 생활/건강      28.481662
대분류_착지물동량 스포츠/레저      2.042624
대분류_착지물동량 식품         39.011246
대분류_착지물동량 출산/육아       1.697392
대분류_착지물동량 패션의류       24.022412
대분류_착지물동량 패션잡화       13.751752
대분류_착지물동량 화장품/미용     30.927057
대분류_착지물동량 가구/인테리어            5
대분류_착지물동량 기타                38
대분류_착지물동량 도서/음반             25
대분류_착지물동량 디지털/가전             9
대분류_착지물동량 생활/건강             28
대분류_착지물동량 스포츠/레저             2
대분류_착지물동량 식품                39
대분류_착지물동량 출산/육아              2
대분류_착지물동량 패션의류              24
대분류_착지물동량 패션잡화              14
대분류_착지물동량 화장품/미용            31
Name: 0, dtype: object

In [118]:
neighborhood_df.isna().sum()

배송년월일                0
자치구명                 0
행정동명                 0
수하인_구코드              0
대분류_착지물동량 가구/인테리어    0
대분류_착지물동량 기타         0
대분류_착지물동량 도서/음반      0
대분류_착지물동량 디지털/가전     0
대분류_착지물동량 생활/건강      0
대분류_착지물동량 스포츠/레저     0
대분류_착지물동량 식품         0
대분류_착지물동량 출산/육아      0
대분류_착지물동량 패션의류       0
대분류_착지물동량 패션잡화       0
대분류_착지물동량 화장품/미용     0
대분류_착지물동량 가구/인테리어    0
대분류_착지물동량 기타         0
대분류_착지물동량 도서/음반      0
대분류_착지물동량 디지털/가전     0
대분류_착지물동량 생활/건강      0
대분류_착지물동량 스포츠/레저     0
대분류_착지물동량 식품         0
대분류_착지물동량 출산/육아      0
대분류_착지물동량 패션의류       0
대분류_착지물동량 패션잡화       0
대분류_착지물동량 화장품/미용     0
dtype: int64

In [116]:
neighborhood_df.to_csv('logistics_dong.csv', index=False, encoding='euc-kr')

In [ ]:
data2 = neighborhood_df.rename(columns={'대분류_착지물동량 가구/인테리어':'가구/인테리어', 
                     '대분류_착지물동량 기타':'기타',
                     '대분류_착지물동량 도서/음반':'도서/음반', 
                     '대분류_착지물동량 디지털/가전':'디지털/가전', 
                     '대분류_착지물동량 생활/건강':'생활/건강',
                     '대분류_착지물동량 스포츠/레저':'스포츠/레저', 
                     '대분류_착지물동량 식품':'식품', 
                     '대분류_착지물동량 출산/육아':'출산/육아', 
                     '대분류_착지물동량 패션의류':'패션의류',
                     '대분류_착지물동량 패션잡화':'패션잡화', 
                     '대분류_착지물동량 화장품/미용':'화장품/미용', 
                     '대분류_착지물동량 가구/인테리어.1':'가구/인테리어r',
                     '대분류_착지물동량 기타.1':'기타r', 
                     '대분류_착지물동량 도서/음반.1':'도서/음반r', 
                     '대분류_착지물동량 디지털/가전.1':'디지털/가전r',
                     '대분류_착지물동량 생활/건강.1':'생활/건강r', 
                     '대분류_착지물동량 스포츠/레저.1':'스포츠/레저r', 
                     '대분류_착지물동량 식품.1':'식품r',
                     '대분류_착지물동량 출산/육아.1':'출산/육아r', 
                     '대분류_착지물동량 패션의류.1':'패션의류r', 
                     '대분류_착지물동량 패션잡화.1':'패션잡화r',
                     '대분류_착지물동량 화장품/미용.1':'화장품/미용r'})

data2.to_csv("logistics_dong2.csv", index=False, encoding='euc-kr')